In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

plt.style.use('ggplot')
import dateutil.parser
import re
import time

%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
datestring = time.strftime("%m-%h-%d")
datestring

'07-Jul-17'

# Functions for the scraper 

In [2]:
#Function to pick every other player because of some duplicates:
def removing_every_other_player(x):
    return x[::2]

In [3]:
#Functions for visiting players profil page. 
def alter(player):
    try:
        return player.find('th', text = re.compile('Alter:')).findNext('td').string
    except:
        return None

In [4]:
def im_team_seit(player):
    try:
        return player.find('th', text = re.compile('Im Team seit:')).findNext('td').string.strip()
    except:
        return None

In [5]:
def groesse(player):
    try:
        return int(player.find('th', text = re.compile('Größe:')).findNext('td').string.replace(',', '').replace(' m', ''))
    except:
        return None

In [6]:
def position(player):
    if player == None:
        return None
    else:
        return player.find('th', text = re.compile('Position:'))\
    .findNext('td').string.strip()

In [7]:
def club(player):
    if player.find('th', text = re.compile('Aktueller Verein:')).string.strip() == 'Aktueller Verein:':
        return player.find('th', text = re.compile('Aktueller')).findNext('td').findNext('a').find('img').get('alt')
    elif player.find('th', text = re.compile('Current club:')).string.strip() == 'Current club:':
        return player.find('th', text = re.compile('Current club:')).findNext('td').findNext('a').find('img').get('alt')
    else:
        return

In [8]:
#Function to extract the player's earlier club
def ex_club(player):
    try:
        return player.find('td', {'class': 'hauptlink no-border-links hide-for-small vereinsname'}).findNext('a').string
    except:
        return None

In [9]:
def agent(player):
    try:
        return player.find('th', text = re.compile("Spielerberater:"))\
    .findNext('td').findNext('a').get('title')
    except:
        return None

In [10]:
def shoes(player):
    try:
        return player.find('th', text = re.compile("Ausrüster:")).findNext('td')\
    .findNext('a').findNext('img').get('title')
    except:
        return None

In [11]:
def marktwert(player):
    try:
        return player.find('div', {'class': 'right-td'}).findNext('a').text.replace(',', '').replace(' Mio. €', '0000').replace(' Tsd. €', '0000')
    except:
        return None

In [12]:
def ausgeliehen_von(player):
    try:
        return player.find('th', text = re.compile("Ausgeliehen von:")).findNext('td').find('a').text
    except:
        return None

In [13]:
def Minuten(x):
    try:
        return int(x.find('td', {'class': 'zentriert'}).find_next('td', {'class': 'rechts'}).string.replace("'", ''))
    except:
        return None

# Scraping

In [14]:
#Header:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [15]:
url = "https://www.transfermarkt.ch/ligue-1/transfers/wettbewerb/FR1/saison_id/2018"
response = requests.get(url, headers=headers)
transfermarkt_soup = BeautifulSoup(response.text, 'html.parser')

In [16]:
transfermarkt = transfermarkt_soup.find_all('a', {'class': 'spielprofil_tooltip'})

In [17]:
#not sure why this playing starts the scraped list. Keep an eye on this
transfermarkt.pop(0)
#Only picking every other player
transfermarkt = removing_every_other_player(transfermarkt)

In [ ]:
transfer_dict = []

for spieler in transfermarkt:
    
    #Name
    Name = spieler.string
    print(Name)
    
    #Link
    spieler_link = spieler['href']
    spieler_link = 'http://www.transfermarkt.ch' + spieler_link
    #print(spieler_link)
    
    #Visiting players profil pages
    spieler_profil = requests.get(spieler_link, headers=headers)
    spieler_profil = BeautifulSoup(spieler_profil.text, 'html.parser')
    #print(spieler_profil)
    
    Alter = alter(spieler_profil)
    #print(Alter)
    Grösse = groesse(spieler_profil)
    #print(Grösse)
    Position = position(spieler_profil)
    #print(Position)
    Verein = club(spieler_profil)
    #print(Verein)
    Ex_Verein = ex_club(spieler_profil)
    #print(Ex_Verein)
    Agent = agent(spieler_profil)
    #print(Agent)
    Ausrüster = shoes(spieler_profil)
    #print(Ausrüster)
    Im_team_seit = im_team_seit(spieler_profil)
    #print(Im_team_seit)
    Marktwert = marktwert(spieler_profil)
    Ausgeliehen_von = ausgeliehen_von(spieler_profil)
    #print(Marktwert)
    #print(Marktwert)
    
    #Visiting Leistungsdaten pages
    spieler_leistungsdaten_link = spieler_link.replace('profil', 'leistungsdaten')
    spieler_leistungsdaten_link = spieler_leistungsdaten_link + '/plus/0?saison=ges'
    spieler_leistungsdaten = requests.get(spieler_leistungsdaten_link, headers=headers)
    spieler_leistungsdaten = BeautifulSoup(spieler_leistungsdaten.text, 'html.parser')
    print(spieler_leistungsdaten_link)
    
    #This is okay to do in a list, as every footballer has these parameters
    Spiele = spieler_leistungsdaten.find('tfoot')
    leistungsdaten = Spiele.find_all('td', {'class': 'zentriert'})
    leistungsdaten_list = []
    for Daten in leistungsdaten:
        Daten = Daten.string
        leistungsdaten_list.append(Daten)
    
    if len(leistungsdaten_list) > 6:
        Totalspiele = int(leistungsdaten_list[0].replace('-', '0'))
        Tore = int(leistungsdaten_list[1].replace('-', '0'))
        Vorlagen = 0
        Gelbe_Karten = int(leistungsdaten_list[2].replace('-', '0'))
        Gelb_Rote_Karten = int(leistungsdaten_list[3].replace('-', '0'))
        Rote_Karten = int(leistungsdaten_list[4].replace('-', '0'))
    
    else:
        Totalspiele = int(leistungsdaten_list[0].replace('-', '0'))
        Tore = int(leistungsdaten_list[1].replace('-', '0'))
        Vorlagen = int(leistungsdaten_list[2].replace('-', '0'))
        Gelbe_Karten = int(leistungsdaten_list[3].replace('-', '0'))
        Gelb_Rote_Karten = int(leistungsdaten_list[4].replace('-', '0'))
        Rote_Karten = int(leistungsdaten_list[5].replace('-', '0'))
        #print(leistungsdaten_list)
    
    #Minuten = Minuten(Spiele)
    
    Spieler_Dict = {"Name": Name,
                    "Alter": Alter,
                    "URL": spieler_link,
                    "Grösse": Grösse,
                    "Position": Position,
                    "Verein": Verein,
                    "Wechsel von": Ex_Verein,
                    "Ausgeliehen von": Ausgeliehen_von,
                    "Agent": Agent,
                    "Ausrüster": Ausrüster,
                    "Im Team seit": Im_team_seit,
                    "Total Spiele": Totalspiele,
                    "Tore": Tore,
                    "Vorlagen": Vorlagen,
                    "Gelbe Karten": Gelbe_Karten,
                    "Gelb-Rote Karten": Gelb_Rote_Karten,
                    "Rote Karten": Rote_Karten,
                    'Marktwert': Marktwert,
                    'Minuten': Minuten
                    }
    
    transfer_dict.append(Spieler_Dict)